In [7]:
import numpy as np
from pylab import *
import cv2
from scipy import signal
import scipy.spatial.distance
from scipy import *
from PIL import Image
import random
import skimage.transform
from skimage import color

AttributeError: type object 'skimage.transform._hough_transform.array' has no attribute '__reduce_cython__'

In [1]:
def warp_images(image0, image1, transform):
    r, c = image1.shape[:2]
    # Note that transformations take coordinates in (x, y) format,
    # not (row, column), in order to be consistent with most literature
    corners = np.array([[0, 0],
                        [0, r],
                        [c, 0],
                        [c, r]])

    # Warp the image corners to their new positions
    warped_corners = transform(corners)

    # Find the extents of both the reference image and the warped
    # target image
    all_corners = np.vstack((warped_corners, corners))

    corner_min = np.min(all_corners, axis=0)
    corner_max = np.max(all_corners, axis=0)

    output_shape = (corner_max - corner_min)
    output_shape = np.ceil(output_shape[::-1])

    offset = skimage.transform.SimilarityTransform(translation=-corner_min)

    image0_ = skimage.transform.warp(image0, offset.inverse, output_shape=output_shape, cval=-1)

    image1_ = skimage.transform.warp(image1, (transform + offset).inverse, output_shape=output_shape, cval=-1)

    image0_zeros = skimage.transform.warp(image0, offset.inverse, output_shape=output_shape, cval=0)

    image1_zeros = skimage.transform.warp(image1, (transform + offset).inverse, output_shape=output_shape, cval=0)

    overlap = (image0_ != -1.0 ).astype(int) + (image1_ != -1.0).astype(int)
    overlap += (overlap < 1).astype(int)
    merged = (image0_zeros+image1_zeros)/overlap

    im = Image.fromarray((255*merged).astype('uint8'), mode='RGB')
    im.save('stitched_images.jpg')
    im.show()

In [ ]:
def main():
    num_match = 1000
    
    left_img = cv2.imread('left.jpg')
    right_img = cv2.imread('right.jpg')
    
    left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    left_sift = cv2.xfeatures2d.SIFT_create()
    left_kp = left_sift.detect(left_gray, None)
    left_img = cv2.drawKeypoints(left_gray, left_kp, left_img)
    cv2.imwrite('sift_left_keypoints.jpg', left_img)
    left_kp, left_des = left_sift.detectAndCompute(left_gray, None)
    
    right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)
    right_sift = cv2.xfeatures2d.SIFT_create()
    right_kp = right_sift.detect(right_gray, None)
    right_img = cv2.drawKeypoints(right_gray, right_kp, right_img)
    cv2.imwrite('sift_right_keypoints.jpg', right_img)
    right_kp, right_des = right_sift.detectAndCompute(right_gray, None)
    
    desp_distances = scipy.spatial.distance.cdist(left_des, right_des, 'sqeuclidean')
    print(desp_distances.shape)
    left_kp = np.array(left_kp)
    right_kp = np.array(right_kp)
    
#     # Compute putative matches
#     match_index = np.argpartition(desp_distances, num_match, axis = -1)[:, :num_match]
#     print(match_index.shape)
#     match_right = match_index.flatten()
#     print(match_right)
#     match_left = [i for i in range(0, desp_distances.shape[0] * num_match)]
#     match_left = np.divide(match_left, num_match)
#     match_left = np.array(match_left, dtype = int)
#     print(match_left)
#     match_left_kp = left_kp[match_left]
#     match_right_kp = right_kp[match_right]
#     match_left_kp = [(int(x.pt[0]), int(x.pt[1])) for x in match_left_kp]
#     match_right_kp = [(int(x.pt[0]), int(x.pt[1])) for x in match_right_kp]
#     match_left_kp = np.asarray(match_left_kp)
#     match_right_kp = np.asarray(match_right_kp)
    

#     flattened_distances = desp_distances.flatten()
#     match_index = np.argpartition(flattened_distances, num_match)[:num_match]
#     match_left = match_index // desp_distances.shape[1]
#     match_right = match_index % desp_distances.shape[1]
#     match_left_kp = left_kp[match_left]
#     match_right_kp = right_kp[match_right]
#     match_left_kp = [(int(x.pt[0]), int(x.pt[1])) for x in match_left_kp]
#     match_right_kp = [(int(x.pt[0]), int(x.pt[1])) for x in match_right_kp]
#     match_left_kp = np.asarray(match_left_kp)
#     match_right_kp = np.asarray(match_right_kp)

    desp1_idx = np.where(desp_distances < 35000)[0]
    desp2_idx = np.where(desp_distances < 35000)[1]
    match_left_kp = np.array([left_kp[idx].pt for idx in desp1_idx])
    match_right_kp = np.array([right_kp[idx].pt for idx in desp2_idx])
    
    left_img = cv2.imread('left.jpg')
    right_img = cv2.imread('right.jpg')
    putative_matches = []
    for i in range(len(match_left_kp)):
        putative_matches.append([match_left_kp[i], match_right_kp[i]])
    draw_matched_inliers(len(match_left_kp), putative_matches, left_img, right_img)
    
    # RANSAC
    max_inlier_count = 0
    num_sample = 4
    for i in range(10000):
        choice = random.sample(range(0, len(match_left_kp)), num_sample)
        left_choice = match_left_kp[choice]
        right_choice = match_right_kp[choice]
        
        # Find A
        A = np.zeros((num_sample * 2, 9))
        for j in range(num_sample):
            x_i = [left_choice[j][0], left_choice[j][1], 1]
            y_i = right_choice[j]
            y_i_x_i = np.multiply(-y_i[1], x_i)
            x_i_x_i = np.multiply(-y_i[0], x_i)
            A[2 * j, 3:6] = x_i
            A[2 * j, 6:9] = y_i_x_i
            A[2 * j + 1, 0:3] = x_i
            A[2 * j + 1, 6:9] = x_i_x_i
        
        # Solve for h in Ah = 0
        U, s, V = np.linalg.svd(A)
        result = V[len(V) - 1]
        result = result.reshape((3, 3))
        result /= result[2, 2]
        if(np.linalg.matrix_rank(result) < 3):
            continue
#         print(result)
        
        # Find number of inliers
        inlier_count = 0
        inliers = []
        for match in range(len(match_left_kp)):
            x = match_left_kp[match][0]
            y = match_left_kp[match][1]
            left_vect = [x, y, 1]
            transformed_vect = np.matmul(result, left_vect)
            transformed_vect = transformed_vect / transformed_vect[2]
            transformed_vect = transformed_vect[:2]
#             print(transformed_vect, match_right_kp[match])
            sqr_dist = transformed_vect - match_right_kp[match]
            sqr_dist = np.linalg.norm(sqr_dist) ** 2
#             print(sqr_dist)
            if sqr_dist < 0.5:
                inlier_count += 1
                inliers.append([match_left_kp[match], match_right_kp[match]])
        if inlier_count > max_inlier_count:
            A = np.zeros((2 * inlier_count, 9))
            for i in range(inlier_count):
                x_i = [inliers[i][0][0], inliers[i][0][1], 1]
                y_i = inliers[i][1]
                y_i_x_i = np.multiply(-y_i[1], x_i)
                x_i_x_i = np.multiply(-y_i[0], x_i)
                A[2 * i, 3:6] = x_i
                A[2 * i, 6:9] = y_i_x_i
                A[2 * i + 1, 0:3] = x_i
                A[2 * i + 1, 6:9] = x_i_x_i
        
            # Solve for h in Ah = 0
            U, s, V = np.linalg.svd(A)
            result = V[len(V) - 1]
            result = result.reshape((3, 3))
            result /= result[2, 2]
            
            inlier_count = 0
            inliers = []
            for match in range(len(match_left_kp)):
                x = match_left_kp[match][0]
                y = match_left_kp[match][1]
                left_vect = [x, y, 1]
                transformed_vect = np.matmul(result, left_vect)
                transformed_vect = transformed_vect / transformed_vect[2]
                transformed_vect = transformed_vect[:2]
#             print(transformed_vect, match_right_kp[match])
                sqr_dist = transformed_vect - match_right_kp[match]
                sqr_dist = np.linalg.norm(sqr_dist) ** 2
#             print(sqr_dist)
                if sqr_dist < 20:
                    inlier_count += 1
                    inliers.append([match_left_kp[match], match_right_kp[match]])
            if inlier_count > max_inlier_count:
                max_inlier_count = inlier_count
                max_inlier = inliers.copy()
                max_result = result.copy()
            
    print(max_inlier_count)


    # Fit using inliers
    inlier_count = max_inlier_count
    print(inlier_count)
    result = max_result
    inliers = max_inlier
#     A = np.zeros((2 * inlier_count, 9))
#     for i in range(inlier_count):
#         x_i = [inliers[i][0][0], inliers[i][0][1], 1]
#         y_i = inliers[i][1]
#         y_i_x_i = np.multiply(-y_i[1], x_i)
#         x_i_x_i = np.multiply(-y_i[0], x_i)
#         A[2 * i, 3:6] = x_i
#         A[2 * i, 6:9] = y_i_x_i
#         A[2 * i + 1, 0:3] = x_i
#         A[2 * i + 1, 6:9] = x_i_x_i
        
#     # Solve for h in Ah = 0
#     U, s, V = np.linalg.svd(A)
#     result = V[len(V) - 1]
#     result = result.reshape((3, 3))
#     result /= result[2, 2]
#     print(result)
    
    transform = skimage.transform.ProjectiveTransform(result)
    left_img = cv2.imread('left.jpg')
    right_img = cv2.imread('right.jpg')
    
    draw_matched_inliers(inlier_count, inliers, left_img, right_img)
    
    warp_images(right_img, left_img, transform)
    
    

if __name__ == "__main__":
    main()

In [3]:
def draw_matched_inliers(max_inlier, best_inliers, orig_img1, orig_img2):
    # build matches
    matches = []
    inlier_kp1 = []
    inlier_kp2 = []
    for i in range(max_inlier):
        kp1 = cv2.KeyPoint()
        kp1.pt = (best_inliers[i][0][0], best_inliers[i][0][1])
        inlier_kp1.append(kp1)
        kp2 = cv2.KeyPoint()
        kp2.pt = (best_inliers[i][1][0], best_inliers[i][1][1])
        inlier_kp2.append(kp2)
        match = cv2.DMatch()
        match.imgIdx, match.queryIdx, match.trainIdx = i, i, i
        matches.append(match)
        
    # Draw lines
    # Reference: https://www.snip2code.com/Snippet/443281/Python-replacement-for-cv2-drawMatches()
    if len(orig_img1.shape) == 3:
        new_shape = (max(orig_img1.shape[0], orig_img2.shape[0]), orig_img1.shape[1]+orig_img2.shape[1], orig_img1.shape[2])
    elif len(orig_img1.shape) == 2:
        new_shape = (max(orig_img1.shape[0], orig_img2.shape[0]), orig_img1.shape[1]+orig_img2.shape[1])
    new_img = np.zeros(new_shape, type(orig_img1.flat[0]))  
    # Place images onto the new image.
    new_img[0:orig_img1.shape[0],0:orig_img1.shape[1]] = orig_img1
    new_img[0:orig_img2.shape[0],orig_img1.shape[1]:orig_img1.shape[1]+orig_img2.shape[1]] = orig_img2
    
    # Draw lines between matches.  Make sure to offset kp coords in second image appropriately.
    r = 10
    thickness = 2
    c = (0, 255, 0)
    for m in matches:
        # Generate random color for RGB/BGR and grayscale images as needed.
        if not color: 
            c = np.random.randint(0,256,3) if len(orig_img1.shape) == 3 else np.random.randint(0,256)
        # So the keypoint locs are stored as a tuple of floats.  cv2.line(), like most other things,
        # wants locs as a tuple of ints.
        end1 = tuple(np.round(inlier_kp1[m.trainIdx].pt).astype(int))
        end2 = tuple(np.round(inlier_kp2[m.queryIdx].pt).astype(int) + np.array([orig_img1.shape[1], 0]))
        cv2.line(new_img, end1, end2, c, thickness)
        cv2.circle(new_img, end1, r, c, thickness)
        cv2.circle(new_img, end2, r, c, thickness)
    
    plt.figure(figsize=(15,15))
    plt.imshow(new_img)
    plt.title('Matched Inliers')
    plt.savefig('part1_matched_inliers.jpg')
    plt.show()
    return